# Program Setup

In [1]:
# Import Libraries and Tools
import os
import sys
from astropy.io import fits
import sunpy.map
from aiapy.calibrate import register, update_pointing
from reproject import reproject_interp
import warnings
warnings.filterwarnings("ignore")
import glob
import numpy as np
import matplotlib.pyplot as plt
import skimage.morphology

# ACWE utilities
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import ACWE Tools
sys.path.append(ROOT_DIR)
from ACWE_python_fall_2023 import acweSaveSeg_v5, acweRestoreScale
from DatasetTools import DataManagmentTools as dmt

plt.rcParams.update({'image.interpolation': 'None'})

In [ ]:
# Carrington Rotation
CR = 'CR2133'

# Datasets
dataFolder    = '/home/jgra/Coronal Holes/newDataset/'
GeneralFolder = '/mnt/coronal_holes/Paper 2/Code 02 Observations/'
evolveFolder  = GeneralFolder + 'EUV_SingleChannel/Standard_history/'
uniElvFolder  = GeneralFolder + 'HMI_Experiments/TestEvolutionMethods/Standard_history/'
stats1Folder  = GeneralFolder + 'EUV_SingleChannel/ScaledStats/'
stats8Folder  = GeneralFolder + 'EUV_SingleChannel/StandardStats/'
seeds1Folder  = GeneralFolder + 'EUV_SingleChannel/ScaledSeedStats/'
seeds8Folder  = GeneralFolder + 'EUV_SingleChannel/StandardSeedStats/'

# Seed Counts by file
seed8title = 'Results/Seed8Counts.' + CR + '.npz'

# Weight compare by Seed
seed8toseg8Utitle = 'Results/UniSeed8toseg8U.npz'
seed8toseg8Wtitle = 'Results/UniSeed8toseg8W.npz'

# Save Information
EUVprefix = 'UniEvolve.'
UniPrefix = 'UniOnUniEvolve.'
folder    = 'EvolveStats/'

# Figure Folder
figFolder = '/home/jgra/Coronal Holes/2nd paper/Code 02/Figures/SeedToSegEval/Samples/'
if not os.path.exists(figFolder):
    os.makedirs(figFolder)

In [ ]:
# Get Data
hmi    = sorted(glob.glob(dataFolder   + CR + '/*/hmi.*.fits'))
aia    = sorted(glob.glob(dataFolder   + CR + '/*/*.193.*.fits'))
orig   = sorted(glob.glob(evolveFolder + CR + '/*/*.npz'))
uni    = sorted(glob.glob(uniElvFolder + CR + '/*/*.npz'))
stats8 = sorted(glob.glob(stats8Folder + CR + '/*/*.npz'))
seeds8 = sorted(glob.glob(seeds8Folder + CR + '/*/*.npz'))

In [ ]:
# Load Number of Seeds/Image
data  = np.load(seed8title, allow_pickle=True)
lst   = data.files
nbrSeed8 = data[lst[0]]

# Load Unweighted Compair 
data  = np.load(seed8toseg8Utitle, allow_pickle=True)
lst   = data.files
seed8toseg8U = data[lst[0]]

# Load Weighted Compair
data  = np.load(seed8toseg8Wtitle, allow_pickle=True)
lst   = data.files
seed8toseg8W = data[lst[0]]

# Usefull Functions

In [ ]:
# Open EUV Image
def openAIA(filename):
    
    # Extract Image and Header Data
    hdulist = fits.open(filename)
    hdulist.verify('silentfix') # no clue why this is needed for successful data read
    h = hdulist[1].header
    J = hdulist[1].data
    hdulist.close()
    
    # Update to Level 1.5 Data Product
    if h['LVL_NUM'] < 1.5:
        m = sunpy.map.Map((J,h))    # Create Sunpy Map
        m = update_pointing(m)      # Update Header based on Latest Information
        m_registrered = register(m) # Recenter and rotate to Solar North
        I = m_registrered.data
        # Undo Keword Renaming
        H = dict()
        for k in m_registrered.meta.keys(): 
            H[k.upper()] = m_registrered.meta[k] 
    # Skip if already Level 1.5
    else:
        I = J*1
        H = dict(h)
        
    # Prepare Display Version
    Idsp = np.clip(I,20,2500)
    Idsp = np.log10(Idsp)
    Idsp = Idsp - np.min(Idsp)
    Idsp = Idsp/np.max(Idsp)
    
    return I,Idsp,H

# Generate Lists

The metric that is being used here is a unipolarity metric wherein 0 indicates perfectly unipolar and 1 indicates perfectly bipolar. Our goal with this work is to identify cases where the the seed reported incorrect unipolarity, so that we can then study its evolution and determin when the reported unipolarity is actually reliable, allowing us to beter eliminate CHs during the evolution process. To do this we will take the diffrence between the reported unipolarity of the seed and the reported unipolarity of the final CH. Ideally, cases where the seed and final CH disagree on whether or not the region is a CH or filiment should have the largest abosolute diffrence between them. Cases where the seed reported CH when the region was a filimet should have a negative (-) sign, while cases where the seed reported a filiment and the region was a CH should have a positive (+) sign. Thus we can take the mimimum and maximum of the ensemble respecitvly to find both cases, meanwhile the index, alongside the number of seeds/image data (`nbrseed8`) should help us identify which file these cases belong to so that we can then track the evolution of the desired groups.

In [ ]:
# Calculate Difference
unweightedDifference = seed8toseg8U[:,0] - seed8toseg8U[:,1]
weightedDifference   = seed8toseg8W[:,0] - seed8toseg8W[:,1]

# Organize
sortUnweight = sorted(unweightedDifference)
sortWeight   = sorted(weightedDifference)

In [ ]:
# Choose sample sets

# Number of Samples
size = len(sortUnweight)

# Placeholders
indexUnotCH = np.empty(size); indexUnotCH[:] = np.nan
indexWnotCH = np.empty(size); indexWnotCH[:] = np.nan
indexUisCH  = np.empty(size); indexUisCH[:]  = np.nan
indexWisCH  = np.empty(size); indexWisCH[:]  = np.nan

# Get Samples
for i in range(size):
    
    # Not CH Cases, requested number of cases, organized from Worst to best
    indexUnotCH[i] = np.where(unweightedDifference==sortUnweight[i])[0][0]
    indexWnotCH[i] = np.where(weightedDifference  ==sortWeight[i]  )[0][0]
    
    # Is CH Cases, requested number of cases, organized from worst to best
    indexUisCH[i]  = np.where(unweightedDifference==sortUnweight[len(sortUnweight)-i-1])[0][0]
    indexWisCH[i]  = np.where(weightedDifference  ==sortWeight[len(sortUnweight)  -i-1])[0][0]

In [ ]:
# Placeholders
imageIndexUnotCH = np.empty(size); imageIndexUnotCH[:] = np.nan
imageIndexWnotCH = np.empty(size); imageIndexWnotCH[:] = np.nan
imageIndexUisCH  = np.empty(size); imageIndexUisCH[:]  = np.nan
imageIndexWisCH  = np.empty(size); imageIndexWisCH[:]  = np.nan

# Find Index of images
seedCumsum = np.cumsum(nbrSeed8)

for i in range(size):
    
    imageIndexUnotCH[i] = np.where(seedCumsum>indexUnotCH[i])[0][0]
    imageIndexWnotCH[i] = np.where(seedCumsum>indexWnotCH[i])[0][0]
    imageIndexUisCH[i]  = np.where(seedCumsum>indexUisCH[i])[0][0]
    imageIndexWisCH[i]  = np.where(seedCumsum>indexWisCH[i])[0][0]

# Convert to Integer Array
imageIndexUnotCH = imageIndexUnotCH.astype(int)
imageIndexWnotCH = imageIndexWnotCH.astype(int)
imageIndexUisCH  = imageIndexUisCH.astype(int)
imageIndexWisCH  = imageIndexWisCH.astype(int)

In [ ]:
# Reset seed Index
indexUnotCH = indexUnotCH - seedCumsum[imageIndexUnotCH - 1]
indexWnotCH = indexWnotCH - seedCumsum[imageIndexWnotCH - 1]
indexUisCH  = indexUisCH  - seedCumsum[imageIndexUisCH  - 1]
indexWisCH  = indexWisCH  - seedCumsum[imageIndexWisCH  - 1]

# Underflow Case
indexUnotCH[np.where(indexUnotCH < 0)] = indexUnotCH[np.where(indexUnotCH < 0)] + seedCumsum[-1]
indexWnotCH[np.where(indexWnotCH < 0)] = indexWnotCH[np.where(indexWnotCH < 0)] + seedCumsum[-1]
indexUisCH[np.where(indexUisCH < 0)]   = indexUisCH[np.where(indexUisCH < 0)]   + seedCumsum[-1]
indexWisCH[np.where(indexWisCH < 0)]   = indexWisCH[np.where(indexWisCH < 0)]   + seedCumsum[-1]

# Convert to Integer Array
indexUnotCH = indexUnotCH.astype(int)
indexWnotCH = indexWnotCH.astype(int)
indexUisCH  = indexUisCH.astype(int)
indexWisCH  = indexWisCH.astype(int)

# Plot Magnetic Filed Data

## Seed FP

In [ ]:
for i in range(25):
    
    # File Names
    titleObs = os.path.basename(orig[imageIndexUnotCH[i]]).split('.npz')[0].split('ACWE.')[1]
    titleEUV = folder + EUVprefix + str(imageIndexUnotCH[i]) + '.' + str(indexUnotCH[i]) + '.npz'
    titleUni = folder + UniPrefix + str(imageIndexUnotCH[i]) + '.' + str(indexUnotCH[i]) + '.npz'
    
    # Open EUV image
    print(titleObs)
    titleObs = glob.glob(dataFolder   + CR + '/*/' + titleObs)[0]
    _,_,H = openAIA(titleObs)
    
    # Open EUV only Seg
    data = np.load(titleEUV, allow_pickle=True)
    lst = data.files
    
    # Separate into Header and Image
    EUVuniU = data[lst[0]]
    EUVuniW = data[lst[1]]
    EUVarea = data[lst[2]]
    
    # Open with Uni Seg
    data = np.load(titleUni, allow_pickle=True)
    lst = data.files
    
    # Separate into Header and Image
    UniUniU = data[lst[0]]
    UniUniW = data[lst[1]]
    UniArea = data[lst[2]]
    
    # Plot
    title = H['T_REC'] + '\nSeed Group ' + str(indexUnotCH[i] + 1)
    print(title)
    
    plt.figure(figsize=[16,7])
    plt.rcParams.update({'font.size': 15})
    plt.suptitle(title)
    plt.subplot(1,2,1)
    plt.plot(EUVuniU,label='ACWE, unweighted')
    plt.plot(EUVuniW,label='ACWE, weighted')
    plt.plot(UniUniU,label='QUACK, unweighted')
    plt.plot(UniUniW,label='QUACK, weighted')
    plt.legend()
    plt.title('Unipolarity vs Iteration')
    plt.xlabel('Number of Iterations')
    plt.ylabel('Unipolarity')
    plt.grid()
    plt.ylim(-0.05,1.05)
    
    plt.subplot(1,2,2)
    plt.plot(EUVarea,label='ACWE')
    plt.plot(UniArea,label='QUACK')
    plt.legend()
    plt.xlabel('Number of Iterations')
    plt.ylabel('Area in Pixels')
    plt.title('Area vs Iteration')
    plt.grid()
    title = 'FPcase' + str(i+1) + 'QUACK.eps'
    plt.savefig(figFolder + title)
    plt.show()
    
    print()

## Seed FN

In [ ]:
for i in range(25):
    
    # File Names
    titleObs = os.path.basename(orig[imageIndexUisCH[i]]).split('.npz')[0].split('ACWE.')[1]
    titleEUV = folder + EUVprefix + str(imageIndexUisCH[i]) + '.' + str(indexUisCH[i]) + '.npz'
    titleUni = folder + UniPrefix + str(imageIndexUisCH[i]) + '.' + str(indexUisCH[i]) + '.npz'
    
    # Open EUV image
    print(titleObs)
    titleObs = glob.glob(dataFolder   + CR + '/*/' + titleObs)[0]
    _,_,H = openAIA(titleObs)
    
    # Open EUV only File
    data = np.load(titleEUV, allow_pickle=True)
    lst = data.files
    
    # Separate into Header and Image
    EUVuniU = data[lst[0]]
    EUVuniW = data[lst[1]]
    EUVarea = data[lst[2]]
    
    # Open with Uni File
    data = np.load(titleUni, allow_pickle=True)
    lst = data.files
    
    # Separate into Header and Image
    UniUniU = data[lst[0]]
    UniUniW = data[lst[1]]
    UniArea = data[lst[2]]
    
    # Plot
    title = H['T_REC'] + '\nSeed Group ' + str(indexUisCH[i] + 1)
    print(title)
    
    plt.figure(figsize=[16,7])
    plt.rcParams.update({'font.size': 15})
    plt.suptitle(title)
    plt.subplot(1,2,1)
    plt.plot(EUVuniU,label='ACWE, unweighted')
    plt.plot(EUVuniW,label='ACWE, weighted')
    plt.plot(UniUniU,label='QUACK, unweighted')
    plt.plot(UniUniW,label='QUACK, weighted')
    plt.legend()
    plt.xlabel('Number of Iterations')
    plt.ylabel('Unipolarity')
    plt.title('Unipolarity vs Iteration')
    plt.grid()
    plt.ylim(-0.05,1.05)
    
    plt.subplot(1,2,2)
    plt.plot(EUVarea,label='ACWE')
    plt.plot(UniArea,label='QUACK')
    plt.legend()
    plt.xlabel('Number of Iterations')
    plt.ylabel('Area in Pixels')
    plt.title('Area vs Iteration')
    plt.grid()
    title = 'FNcase' + str(i + 1) + 'QUACK.eps'
    plt.savefig(figFolder + title)
    plt.show()
    
    print()